In [1]:
import sys
import os

project_root = os.path.abspath("..")
if project_root not in sys.path:
    sys.path.insert(0, project_root)

In [ ]:
# ========================= 0. Import =========================
import os, cv2, numpy as np, torch, matplotlib.pyplot as plt
from torchvision import datasets, transforms
from torch.utils.data import ConcatDataset
from collections import Counter
from sklearn.neighbors import KernelDensity   # KDE
from tqdm import tqdm

# ========================= 1. Thông số =========================
DATA_DIR = "../dataset/Soil"
IMG_SIZE   = 64
AUG_FACTOR = 5
BINS       = 64           # vẫn giữ 32 “bin” để trực quan
N_JOBS     = -1           # dùng hết CPU cho KDE

# ========================= 2. Augmentation =========================
train_tf = transforms.Compose([
    transforms.RandomResizedCrop(IMG_SIZE, scale=(0.7, 1.0)),
    transforms.RandomRotation(degrees=30),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
   #  transforms.GaussianBlur(kernel_size=5, sigma=(0.1, 2.0)),
    transforms.ToTensor(),                               # về [0,1]
   #  transforms.Normalize(mean=[0.485, 0.456, 0.406],
   #                       std =[0.229, 0.224, 0.225])
])

# ========================= 3. Dataset =========================
original_ds = datasets.ImageFolder(DATA_DIR)
original_counts = Counter([lbl for _, lbl in original_ds])
print("📊 Số lượng mẫu GỐC theo lớp:")
for cls, cnt in original_counts.items():
    print(f"  {original_ds.classes[cls]}: {cnt}")

augmented_ds = ConcatDataset(
    [datasets.ImageFolder(DATA_DIR, transform=train_tf) for _ in range(AUG_FACTOR)]
)
